In [1]:
from IPython.display import clear_output 
import cv2
import numpy as np
import mediapipe as mp
import time
import math

In [2]:
def pythagorean_theorem_dot_dist(coordinate_list):
    dot_distance = math.sqrt((coordinate_list[0].x - coordinate_list[1].x)**2 + (coordinate_list[0].y - coordinate_list[1].y)**2 + (coordinate_list[0].z - coordinate_list[1].z)**2)
    return dot_distance

def hand_detection(video_path : str) :
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands
    
    cap = cv2.VideoCapture(0)
    
    middle_finger_mcp_list = []
    
    middle_finger_mcp = 0.015787780284881592
    
    with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        no_detection_time = None  # 감지되지 않는 시간을 초기화
        not_moved_time = None
        previous_dot_distance = None
        user_score = 100

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("video is end.")
                break

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                
                middle_finger_mcp_list.append(results.multi_hand_landmarks[0].landmark[9])
                
                print(results.multi_hand_landmarks[0].landmark[9])
                
                # 손이 움직이지 않는지 판별하는 코드
                if len(middle_finger_mcp_list) > 2 :
                    dot_distance = pythagorean_theorem_dot_dist(middle_finger_mcp_list)
                    if previous_dot_distance is not None:
                        distance_change = abs(dot_distance - previous_dot_distance)
                        if distance_change <= 0.03:
                            print(distance_change)
                            if not_moved_time is None :
                                not_moved_time = time.time()
                            elif time.time() - not_moved_time > 3 :
                                if user_score > 0 :
                                    user_score -= 2
                                not_moved_time = None
                        else : 
                            not_moved_time = None
                            
                    previous_dot_distance = dot_distance
                    middle_finger_mcp_list.pop(1)
                
                for hand_landmarks in results.multi_hand_landmarks:    
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                no_detection_time = None  # 손이 감지되면 감지되지 않는 시간을 초기화
            else:
                if no_detection_time is None:
                    no_detection_time = time.time()  # 손이 감지되지 않는 시간을 기록 시작
                elif time.time() - no_detection_time > 2 :  # 손이 감지되지 않는 시간이 5초 이상 지나면
                    if user_score > 0 :
                        user_score -= 2
                    no_detection_time = None
            clear_output(wait=True)
            print(user_score)
            
            cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

In [3]:
hand_detection("video/hand2.mp4")

100
